In [1]:
import sys
sys.path.append("/pod/2/ke-lab/LUOZ/Singularity/iM6A")

In [2]:
from keras.models import load_model
from pkg_resources import resource_filename
import numpy as np
import pandas as pd
from Bio.Seq import Seq
import keras.backend as kb

Using TensorFlow backend.


In [3]:
def one_hot_encode(seq):

    map = np.asarray([[0, 0, 0, 0],
                      [1, 0, 0, 0],
                      [0, 1, 0, 0],
                      [0, 0, 1, 0],
                      [0, 0, 0, 1]])

    seq = seq.upper().replace('A', '\x01').replace('C', '\x02')
    seq = seq.replace('G', '\x03').replace('T', '\x04').replace('N', '\x00')

    return map[np.fromstring(seq, np.int8) % 5]

In [4]:
def categorical_crossentropy_2d(y_true, y_pred):
    # Standard categorical cross entropy for sequence outputs

    return - kb.mean(y_true[:, :, 0]*kb.log(y_pred[:, :, 0]+1e-10)
                   + y_true[:, :, 1]*kb.log(y_pred[:, :, 1]+1e-10))

In [5]:
context = 10000

In [6]:
paths = ('Test/humanRAC10000_c{}.h5'.format(x) for x in range(1, 6))

In [7]:
models = [load_model(resource_filename('m6AAI', x), custom_objects={'categorical_crossentropy_2d': categorical_crossentropy_2d}) for x in paths]

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead


### Read data

In [8]:
Fasta = pd.read_csv("/pod/2/ke-lab/LUOZ/m6AQTL/ClinVar_Fasta.csv", keep_default_na=False)

In [9]:
Fasta

,chrom,SNPID,name,POS,REF,ALT,strand,txStart,txEnd,cdsStart,cdsEnd,LastExonStart,LastExonEnd,Sequence,GeneLength,LastExonLength,SNPposition
0,chr15,rs1596137692,ZNF592,85345253,C,A,+,85291865,85349202,85325906,85345624,85345093,85349202,ctggcctgcagtcgccgccgccgccgccAGGTAAGCGCCCCCCGcg...,57337,4109,53388
1,chr15,rs143259756,ZNF592,85345216,C,T,+,85291865,85349202,85325906,85345624,85345093,85349202,ctggcctgcagtcgccgccgccgccgccAGGTAAGCGCCCCCCGcg...,57337,4109,53351
2,chr11,rs267606830,FOXRED1,126147412,A,G,+,126139027,126148019,126139101,126147584,126147329,126148019,GCGCGGAGCAGTGACGGCTGCGATAATAGCGAGGCAGCAGTGCAGC...,8992,690,8385
3,chr11,rs201261500,FOXRED1,126147375,C,G,+,126139027,126148019,126139101,126147584,126147329,126148019,GCGCGGAGCAGTGACGGCTGCGATAATAGCGAGGCAGCAGTGCAGC...,8992,690,8348
4,chr11,rs863224020,FOXRED1,126147430,G,A,+,126139027,126148019,126139101,126147584,126147329,126148019,GCGCGGAGCAGTGACGGCTGCGATAATAGCGAGGCAGCAGTGCAGC...,8992,690,8403
5,chr11,rs372575072,FOXRED1,126147669,T,C,+,126139027,126148019,126139101,126147584,126147329,126148019,GCGCGGAGCAGTGACGGCTGCGATAATAGCGAGGCAGCAGTGCAGC...,8992,690,8642
6,chr11,rs667627,FOXRED1,126147697,G,A,+,126139027,126148019,126139101,126147584,126147329,126148019,GCGCGGAGCAGTGACGGCTGCGATAATAGCGAGGCAGCAGTGCAGC...,8992,690,8670
7,chr11,rs570233921,FOXRED1,126147743,C,T,+,126139027,126148019,126139101,126147584,126147329,126148019,GCGCGGAGCAGTGACGGCTGCGATAATAGCGAGGCAGCAGTGCAGC...,8992,690,8716
8,chr11,rs185024577,FOXRED1,126148016,T,G,+,126139027,126148019,126139101,126147584,126147329,126148019,GCGCGGAGCAGTGACGGCTGCGATAATAGCGAGGCAGCAGTGCAGC...,8992,690,8989
9,chr11,rs886047983,FOXRED1,126147976,G,A,+,126139027,126148019,126139101,126147584,126147329,126148019,GCGCGGAGCAGTGACGGCTGCGATAATAGCGAGGCAGCAGTGCAGC...,8992,690,8949


### Select positive strand

In [10]:
Positive = Fasta[Fasta["strand"]=="+"]
Positive = Positive.reset_index(drop = True)

In [11]:
Positive

,chrom,SNPID,name,POS,REF,ALT,strand,txStart,txEnd,cdsStart,cdsEnd,LastExonStart,LastExonEnd,Sequence,GeneLength,LastExonLength,SNPposition
0,chr15,rs1596137692,ZNF592,85345253,C,A,+,85291865,85349202,85325906,85345624,85345093,85349202,ctggcctgcagtcgccgccgccgccgccAGGTAAGCGCCCCCCGcg...,57337,4109,53388
1,chr15,rs143259756,ZNF592,85345216,C,T,+,85291865,85349202,85325906,85345624,85345093,85349202,ctggcctgcagtcgccgccgccgccgccAGGTAAGCGCCCCCCGcg...,57337,4109,53351
2,chr11,rs267606830,FOXRED1,126147412,A,G,+,126139027,126148019,126139101,126147584,126147329,126148019,GCGCGGAGCAGTGACGGCTGCGATAATAGCGAGGCAGCAGTGCAGC...,8992,690,8385
3,chr11,rs201261500,FOXRED1,126147375,C,G,+,126139027,126148019,126139101,126147584,126147329,126148019,GCGCGGAGCAGTGACGGCTGCGATAATAGCGAGGCAGCAGTGCAGC...,8992,690,8348
4,chr11,rs863224020,FOXRED1,126147430,G,A,+,126139027,126148019,126139101,126147584,126147329,126148019,GCGCGGAGCAGTGACGGCTGCGATAATAGCGAGGCAGCAGTGCAGC...,8992,690,8403
5,chr11,rs372575072,FOXRED1,126147669,T,C,+,126139027,126148019,126139101,126147584,126147329,126148019,GCGCGGAGCAGTGACGGCTGCGATAATAGCGAGGCAGCAGTGCAGC...,8992,690,8642
6,chr11,rs667627,FOXRED1,126147697,G,A,+,126139027,126148019,126139101,126147584,126147329,126148019,GCGCGGAGCAGTGACGGCTGCGATAATAGCGAGGCAGCAGTGCAGC...,8992,690,8670
7,chr11,rs570233921,FOXRED1,126147743,C,T,+,126139027,126148019,126139101,126147584,126147329,126148019,GCGCGGAGCAGTGACGGCTGCGATAATAGCGAGGCAGCAGTGCAGC...,8992,690,8716
8,chr11,rs185024577,FOXRED1,126148016,T,G,+,126139027,126148019,126139101,126147584,126147329,126148019,GCGCGGAGCAGTGACGGCTGCGATAATAGCGAGGCAGCAGTGCAGC...,8992,690,8989
9,chr11,rs886047983,FOXRED1,126147976,G,A,+,126139027,126148019,126139101,126147584,126147329,126148019,GCGCGGAGCAGTGACGGCTGCGATAATAGCGAGGCAGCAGTGCAGC...,8992,690,8949


In [12]:
Sequence = Positive["Sequence"].tolist()
Ref = Positive["REF"].tolist()
Alt = Positive["ALT"].tolist()
Position = Positive["SNPposition"].tolist()

In [13]:
for i in range(len(Sequence)):
    input_sequence = Sequence[i]
    seq = input_sequence[(Positive.loc[i,("SNPposition")]-1):(Positive.loc[i,("SNPposition")])]
    print(seq)

C
C
A
C
G
T
G
C
T
G
T
C
C
T
T
T
A
G
C
G
T
C
G
C
T
A
T
C
G
T
C
G
C
C
A
C
C
G
C
A
C
A
T
G
G
C
T
C
C
G
T
A
C
G
C
T
A
G
C
A
G
G
C
C
G
T
C
C
T
G
C
G
C
A
C
G
G
G
G
T
G
C
G
G
C
C
G
G
C
G
C
C
A
C
G
A
G
C
G
C
G
C
G
A
A
G
G
A
C
C
C
G
T
C
C
A
T
T
G
C
C
C
G
C
G
A
A
C
G
G
C
A
C
T
C
G
C
A
C
A
A
T
C
C
A
C
G
C
A
C
T
C
C
G
G
C
G
C
C
G
G
G
C
G
C
G
G
G
C
G
A
C
C
G
C
C
A
C
C
G
C
T
C
A
A
A
G
G
A
T
C
C
T
T
C
C
G
G
A
T
G
G
C
C
G
T
C
G
C
T
C
A
C
A
G
A
C
C
C
A
G
C
G
C
G
G
G
C
G
T
G
A
T
G
C
G
G
T
C
G
A
G
C
C
G
C
A
G
C
G
T
C
G
T
T
C
C
C
C
G
C
C
G
C
C
G
C
G
G
G
G
A
T
G
C
G
T
T
G
G
C
G
G
T
G
A
G
T
G
G
C
C
G
G
A
G
G
A
T
G
G
C
C
C
A
T
G
C
C
C
T
T
C
A
G
C
G
G
G
A
G
G
C
G
A
C
C
G
C
G
G
G
A
C
C
T
C
A
A
A
G
G
C
C
C
G
G
A
C
T
T
G
G
G
C
A
T
C
G
A
T
G
C
G
T
T
G
A
A
T
A
G
G
G
C
C
G
G
C
C
C
A
T
G
T
T
C
A
C
G
C
C
A
G
C
T
G
G
C
T
A
G
G
T
T
T
C
T
A
G
A
C
G
G
G
C
C
G
G
C
G
A
A
A
C
T
A
A
C
G
C
T
C
A
A
G
G
C
A
T
G
G
T
G
A
C
G
G
G
A
C
T
C
C
G
G
C
C
G
G
G
T
A
A
C
A
C
C
C
C
A
G
G
C
G
C
A
C
G
C
G
G
A
A
C
C
T
C
A
A
C
A
C
G
C
A
C
C
C
G


A
C
A
C
C
C
G
T
G
A
T
G
C
C
T
G
T
T
C
C
T
G
T
G
G
T
A
A
G
C
G
G
G
G
T
T
G
G
C
C
G
A
C
T
G
G
G
A
G
A
C
T
C
G
G
A
G
G
A
C
T
G
T
C
A
G
G
A
A
C
G
C
A
C
T
C
G
G
C
A
G
A
C
T
G
C
A
G
C
C
C
T
C
T
A
C
C
G
A
G
A
C
T
A
C
C
C
G
G
C
C
C
A
C
C
A
G
T
C
C
C
G
G
G
C
T
C
G
G
C
A
C
G
C
T
A
C
A
A
C
T
C
C
T
G
C
T
T
T
A
G
G
A
C
T
C
A
C
G
C
C
G
C
T
C
T
C
G
T
G
A
C
C
T
G
T
G
T
G
A
C
G
C
T
G
C
T
G
G
C
A
T
A
C
G
G
C
C
G
G
A
A
T
G
G
T
G
C
T
T
C
G
C
G
C
G
T
C
T
G
A
G
C
C
C
C
A
C
G
C
G
G
G
G
G
G
T
G
A
G
A
T
G
C
C
G
C
C
G
C
C
G
A
G
G
C
T
C
C
C
G
T
G
G
C
C
G
G
G
G
G
T
A
C
C
G
T
G
C
G
C
C
A
C
C
A
G
C
A
C
C
C
C
G
A
G
G
C
G
A
G
G
G
C
C
C
C
G
G
C
T
C
G
C
C
A
G
G
T
A
G
G
C
G
T
T
A
G
G
C
T
A
A
C
A
T
T
A
A
A
C
A
A
A
A
C
G
C
G
A
G
G
C
C
G
C
G
G
C
C
G
A
C
G
C
G
C
G
G
G
T
A
T
C
C
A
C
T
G
A
T
C
C
A
A
C
C
G
A
A
C
C
C
T
C
C
C
G
C
G
G
C
T
T
G
A
C
C
C
C
C
C
A
T
C
G
G
C
A
G
C
C
C
G
C
C
C
G
C
G
T
G
G
C
G
C
C
C
A
G
C
G
G
C
G
G
G
G
G
C
G
C
G
C
T
A
G
C
G
G
G
G
C
A
G
C
T
A
C
G
T
C
G
C
T
G
G
T
C
G
G
G
C
G
C
C
G
C
C
T
C
C
C
G
C
C
C
C
G
C


T
C
T
G
A
C
G
C
C
A
T
G
C
C
G
C
G
G
C
C
T
G
C
G
G
G
G
C
C
A
C
G
A
T
G
A
T
C
T
C
G
C
A
T
G
A
T
T
G
G
C
C
C
G
A
A
T
C
C
C
G
T
T
G
G
A
C
T
C
A
C
C
C
A
G
C
G
G
A
C
T
G
A
A
C
G
G
A
A
G
C
G
G
T
A
A
A
G
G
C
C
T
A
G
G
C
A
G
A
T
C
G
T
A
A
C
G
G
C
C
G
G
A
G
A
C
T
C
C
C
A
A
G
T
G
C
C
G
G
C
A
C
T
G
C
C
C
G
T
A
A
T
T
A
C
A
A
C
G
C
C
C
C
A
C
G
T
A
C
T
C
C
A
A
A
T
C
C
C
T
A
A
C
A
A
G
A
G
G
T
T
T
C
G
C
C
G
C
C
G
C
A
G
G
C
G
G
C
C
A
A
C
T
C
G
G
G
T
G
T
C
C
A
G
G
G
C
G
G
A
T
G
A
G
A
A
G
G
C
G
T
C
G
C
T
C
C
G
C
A
A
C
G
A
T
C
C
A
A
C
C
C
C
C
C
G
G
G
G
G
G
C
G
G
C
C
A
C
C
A
G
A
A
G
G
G
C
C
C
C
C
T
G
C
T
C
A
C
C
G
C
C
A
C
A
T
C
A
G
C
G
A
A
C
G
G
C
C
C
G
C
T
T
C
G
G
A
C
T
C
A
T
G
A
A
A
A
T
C
T
T
A
G
A
T
T
A
G
T
T
A
C
C
C
C
G
A
A
A
T
A
A
A
A
A
T
C
A
C
A
G
G
G
C
C
G
T
T
G
G
T
T
C
C
G
A
T
T
G
T
G
G
C
A
G
G
T
T
A
G
T
T
A
G
C
A
C
C
G
G
G
C
C
C
C
C
G
G
G
A
G
C
C
G
G
C
C
A
G
G
C
C
G
G
G
C
G
A
C
G
C
C
G
C
G
C
A
G
G
C
C
C
C
G
A
G
C
C
G
G
T
G
C
T
G
C
A
C
G
C
G
G
C
T
T
G
C
G
G
G
G
C
G
C
C
G
C
G
T
C
G
T
A
G
T
A
T
T
G
G


G
C
C
A
C
C
T
C
T
T
A
T
C
C
G
C
A
G
A
T
C
G
C
G
A
T
G
G
C
A
G
C
G
T
G
C
A
T
C
G
C
C
G
C
G
C
C
T
G
T
G
C
G
G
C
T
C
C
A
C
T
G
G
C
C
G
G
C
C
A
G
G
T
C
G
T
G
G
C
G
G
C
A
G
A
G
G
C
A
C
T
G
C
G
C
C
C
A
T
A
C
G
C
C
T
T
C
A
C
G
G
T
T
T
A
A
C
T
G
T
G
G
A
C
A
G
T
A
A
C
G
A
C
G
A
T
A
C
A
C
G
C
C
G
T
A
G
C
G
C
G
G
C
G
C
G
C
G
C
G
C
C
C
C
C
C
G
G
C
G
C
C
C
G
C
G
C
C
A
T
A
G
T
G
C
G
C
C
C
C
T
C
C
C
G
G
C
G
G
A
A
G
G
G
C
C
A
T
A
G
A
C
G
T
T
T
G
A
G
G
C
A
C
T
A
A
A
A
C
A
T
G
C
C
G
C
A
G
A
A
T
G
G
C
G
G
A
C
A
A
C
T
C
G
C
G
T
G
T
C
A
C
T
C
A
C
T
G
G
A
A
C
A
G
G
T
T
G
G
C
G
G
C
A
C
G
A
T
G
A
C
C
C
C
A
C
G
G
C
T
G
T
G
C
G
C
T
T
T
G
G
A
A
A
C
T
A
T
G
G
A
T
C
G
C
G
C
G
T
T
G
G
C
G
T
A
G
G
G
G
C
T
A
C
A
T
A
A
G
C
C
A
C
C
T
C
A
A
C
C
G
G
C
G
C
G
G
G
A
G
T
C
C
G
G
T
A
C
G
C
C
C
G
G
G
G
G
T
C
C
T
G
C
G
G
G
G
C
G
A
T
T
T
C
T
A
T
C
G
G
G
C
T
G
C
A
A
C
A
T
C
A
T
G
A
G
A
C
G
A
T
C
C
G
T
G
C
T
T
A
C
A
A
A
C
G
A
C
T
A
G
G
C
T
G
T
C
A
G
G
T
A
C
T
T
C
T
A
C
A
C
C
C
A
G
C
C
C
G
T
G
G
G
A
G
G
T
T
A
T
A
G
G
C
A
G
G
G
G
C


T
T
C
C
G
C
G
C
A
G
G
C
A
G
A
A
G
G
T
G
T
C
G
G
G
A
T
C
C
G
C
T
A
G
A
A
C
T
C
T
A
C
G
G
T
G
A
C
A
C
G
A
G
G
G
C
C
C
C
C
C
G
A
C
C
G
G
C
G
G
C
C
C
G
G
C
A
G
G
G
T
C
G
A
C
C
C
C
G
C
T
A
T
A
T
A
A
A
G
C
A
A
G
A
A
A
G
T
T
C
T
G
A
T
G
A
A
A
C
C
G
C
G
C
G
A
T
C
A
C
T
C
T
T
C
G
A
T
A
T
T
T
G
G
A
C
G
C
G
T
C
C
G
G
C
C
A
G
G
T
C
G
C
C
A
G
C
C
A
G
G
A
T
G
G
G
G
T
C
G
T
G
G
C
C
G
G
A
G
C
G
G
C
A
C
A
G
G
A
C
C
G
G
T
C
T
C
C
G
A
A
G
C
T
C
C
T
G
C
C
C
C
A
G
G
T
C
A
C
C
G
C
C
G
G
C
G
A
G
A
C
G
G
C
T
C
T
T
G
A
C
T
A
G
C
C
C
C
C
C
G
G
G
C
G
G
A
A
G
A
A
C
C
C
C
C
A
G
G
C
C
A
A
C
A
G
A
A
A
T
G
G
G
G
C
T
T
G
A
A
A
C
A
A
G
C
A
G
G
T
T
A
T
T
A
T
G
C
C
C
A
T
G
A
C
T
A
G
A
C
C
C
G
C
C
C
T
C
G
G
T
G
T
C
T
G
T
T
C
C
G
G
A
C
G
G
G
T
G
A
C
G
G
C
G
C
C
C
A
C
C
T
A
A
C
C
C
C
G
G
T
C
A
G
C
C
G
G
T
T
G
A
A
T
C
A
G
T
G
G
C
G
G
A
G
A
C
T
C
A
G
G
G
C
A
A
A
A
T
C
C
G
G
A
T
G
A
A
G
A
C
A
A
A
T
G
T
C
G
A
C
A
G
T
G
G
A
C
C
G
T
G
C
A
G
G
C
C
G
T
A
T
C
T
T
A
G
G
C
G
G
G
C
A
G
G
T
G
G
C
A
G
G
C
C
C
A
A
A
A
T
C
T
C
G
G
A
G
A
T


G
C
C
T
G
A
C
C
T
C
C
T
G
C
C
G
G
G
C
G
C
G
C
A
G
C
T
G
C
C
G
G
G
C
C
C
T
G
G
C
T
G
G
T
C
T
G
C
G
T
G
C
C
G
T
C
C
G
G
C
G
C
G
G
T
G
C
G
G
G
A
G
C
C
C
G
C
C
C
G
G
C
C
G
A
A
C
A
C
C
C
A
G
G
C
T
G
C
G
G
C
C
C
C
G
G
C
C
G
C
G
C
C
C
T
G
C
T
C
G
C
C
G
G
C
C
G
G
A
T
G
G
G
C
A
G
C
C
C
G
T
G
C
G
G
C
G
C
G
A
C
C
G
C
T
G
C
G
T
G
T
G
C
G
G
C
C
G
C
G
G
T
G
C
C
G
G
C
A
C
G
G
G
G
C
T
C
T
G
C
G
C
C
C
A
C
C
A
A
G
C
G
G
G
A
C
A
T
C
T
G
G
A
A
C
A
G
G
A
T
G
T
G
C
A
G
C
C
A
A
G
G
C
G
G
C
G
G
G
C
A
A
A
C
G
C
C
C
C
C
C
G
C
C
T
C
C
G
T
C
G
G
C
C
C
T
G
G
A
G
C
C
C
C
C
C
G
C
C
C
G
A
G
C
C
C
A
A
A
C
A
C
C
A
G
A
G
G
G
C
T
A
C
G
C
G
G
G
G
C
C
C
G
C
C
C
C
C
C
C
C
C
G
C
C
A
A
G
A
G
G
A
G
G
T
G
A
G
C
T
A
A
C
A
G
C
C
G
A
G
T
A
A
A
C
A
T
A
T
A
G
T
A
A
A
C
G
G
G
G
T
G
A
A
G
G
C
G
G
C
C
G
C
C
G
C
G
C
C
G
C
A
A
C
G
G
C
C
G
A
A
C
C
T
G
A
T
C
C
G
C
G
G
G
G
C
G
G
C
G
G
G
C
G
A
C
G
C
G
G
G
C
C
C
C
C
C
C
C
T
A
C
A
G
A
C
C
C
G
C
G
C
T
C
C
G
C
C
G
G
G
G
C
G
C
C
G
G
A
G
T
A
G
C
A
C
C
C
G
C
G
G
C
C
G
G
C
C
C
G
C
C
G
G
A
C
T
C
T
C


C
T
T
C
G
T
C
C
C
G
G
C
C
C
C
G
C
G
G
G
A
C
T
C
G
G
C
C
C
A
T
C
C
C
G
G
G
C
G
T
G
C
C
G
C
A
G
G
C
G
G
G
G
T
C
G
G
G
A
C
T
A
G
A
C
C
G
A
G
G
G
G
C
G
C
G
G
G
C
C
T
G
G
C
G
G
C
C
G
G
A
G
C
G
A
A
T
G
A
A
C
G
G
C
G
G
G
G
G
G
T
G
G
G
C
T
T
G
T
G
G
C
C
G
A
C
G
C
G
C
C
C
G
C
A
C
C
A
T
G
G
C
C
A
C
C
C
C
C
G
C
C
G
A
C
G
G
C
G
G
G
A
G
T
C
G
A
C
C
C
A
A
G
C
G
C
G
T
C
G
C
G
C
C
A
G
C
G
T
G
G
T
G
G
T
T
A
G
A
A
G
C
C
G
C
C
A
C
C
C
C
G
G
T
G
G
C
C
C
G
A
A
G
A
C
G
C
G
T
A
G
T
C
C
C
T
C
T
G
T
C
G
G
T
G
C
C
C
C
G
C
G
G
G
A
G
T
C
A
G
T
A
A
C
T
C
A
G
C
C
C
G
A
C
C
C
C
G
A
G
G
T
T
G
G
T
G
T
T
G
G
T
G
C
C
G
T
C
A
C
C
G
T
C
T
C
C
G
C
C
C
C
C
C
C
C
C
G
C
T
A
T
C
G
C
A
C
C
G
C
G
G
G
C
G
C
C
C
C
G
G
C
G
C
C
C
C
G
C
C
C
G
C
C
C
C
C
C
C
A
C
G
C
C
C
T
G
C
C
C
A
G
G
C
T
A
G
T
G
G
C
A
G
C
G
A
G
G
C
G
C
A
C
A
C
C
G
A
C
G
A
G
C
G
C
G
C
G
T
T
C
C
C
A
C
C
G
C
G
C
G
C
C
A
C
A
G
C
G
C
G
C
G
C
A
T
T
G
C
A
C
C
C
G
C
C
T
C
T
C
T
C
C
C
G
T
T
C
C
A
C
C
C
C
G
A
C
G
C
G
G
C
G
G
T
G
G
T
G
G
C
C
A
A
C
C
G
C
C
C
G
C
C
G
G
C
A
A
G
C


In [14]:
for i in range(len(Sequence)):
    seq = Sequence[i]
    input_sequence = seq[:(Positive.loc[i,("SNPposition")]-1)] + Alt[i] + seq[(Positive.loc[i,("SNPposition")]):]
    
    x = one_hot_encode('N'*(context//2) + input_sequence + 'N'*(context//2))[None, :]
    y = np.mean([models[m].predict(x) for m in range(5)], axis=0)
    m6AAI_prob = y[0, :, 1]
    Probability = pd.DataFrame({'Probability':m6AAI_prob})
    
    df = pd.DataFrame(np.random.randn((Positive.loc[i,"GeneLength"]), 3))
    df.columns = ["name", "chrom", "strand"]
    df["name"] = Positive.loc[i,"SNPID"]
    df["chrom"] = Positive.loc[i,"chrom"]
    df["strand"] = Positive.loc[i,"strand"]
    
    list = range(Positive.loc[i,"txStart"], Positive.loc[i,"txEnd"])
    Start = pd.DataFrame(list, columns=["Start"])
    df = pd.concat([df, Start], axis=1)
    df["End"] = df["Start"]
    df = pd.concat([df, Probability], axis=1)
    df = df.tail(Positive.loc[i,"LastExonLength"])    
    df.columns = ["name", "chrom", "strand", "Start", "End", "Probabilty"]
    df.to_csv("./m6AQTL/{}.bed".format(Positive.loc[i,"SNPID"]), sep="\t", index=False)


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  if sys.path[0] == '':


### Select negative sample

In [15]:
Negative = Fasta[Fasta["strand"]=="-"]
Negative = Negative.reset_index(drop = True)

In [16]:
Negative

,chrom,SNPID,name,POS,REF,ALT,strand,txStart,txEnd,cdsStart,cdsEnd,LastExonStart,LastExonEnd,Sequence,GeneLength,LastExonLength,SNPposition
0,chr2,rs771077097,WDR35,20113460,C,T,-,20110020,20189892,20113318,20189776,20110020,20113469,TTATTTTTTTCTTCAATGTTTATTATTTTCGTATTTTTTCACTTAA...,79872,3449,76433
1,chr2,rs147325795,WDR35,20113365,A,C,-,20110020,20189892,20113318,20189776,20110020,20113469,TTATTTTTTTCTTCAATGTTTATTATTTTCGTATTTTTTCACTTAA...,79872,3449,76528
2,chr2,rs113280329,WDR35,20110051,G,A,-,20110020,20189892,20113318,20189776,20110020,20113469,TTATTTTTTTCTTCAATGTTTATTATTTTCGTATTTTTTCACTTAA...,79872,3449,79842
3,chr2,rs886055392,WDR35,20110330,G,A,-,20110020,20189892,20113318,20189776,20110020,20113469,TTATTTTTTTCTTCAATGTTTATTATTTTCGTATTTTTTCACTTAA...,79872,3449,79563
4,chr2,rs565144558,WDR35,20110700,G,A,-,20110020,20189892,20113318,20189776,20110020,20113469,TTATTTTTTTCTTCAATGTTTATTATTTTCGTATTTTTTCACTTAA...,79872,3449,79193
5,chr2,rs751743368,WDR35,20110760,T,C,-,20110020,20189892,20113318,20189776,20110020,20113469,TTATTTTTTTCTTCAATGTTTATTATTTTCGTATTTTTTCACTTAA...,79872,3449,79133
6,chr2,rs112569580,WDR35,20110911,C,G,-,20110020,20189892,20113318,20189776,20110020,20113469,TTATTTTTTTCTTCAATGTTTATTATTTTCGTATTTTTTCACTTAA...,79872,3449,78982
7,chr2,rs757203175,WDR35,20111737,G,A,-,20110020,20189892,20113318,20189776,20110020,20113469,TTATTTTTTTCTTCAATGTTTATTATTTTCGTATTTTTTCACTTAA...,79872,3449,78156
8,chr2,rs886055400,WDR35,20111858,T,G,-,20110020,20189892,20113318,20189776,20110020,20113469,TTATTTTTTTCTTCAATGTTTATTATTTTCGTATTTTTTCACTTAA...,79872,3449,78035
9,chr2,rs368665906,WDR35,20112066,G,C,-,20110020,20189892,20113318,20189776,20110020,20113469,TTATTTTTTTCTTCAATGTTTATTATTTTCGTATTTTTTCACTTAA...,79872,3449,77827


In [17]:
Sequence = Negative["Sequence"].tolist()
Ref = Negative["REF"].tolist()
Alt = Negative["ALT"].tolist()
Position = Negative["SNPposition"].tolist()

In [18]:
for i in range(len(Sequence)):
    input_sequence = Sequence[i][::-1]
    input_sequence = Seq(input_sequence).complement()
    input_sequence = str(input_sequence)    
    seq = input_sequence[(Negative.loc[i,("SNPposition")]-1):(Negative.loc[i,("SNPposition")])]
    print(seq)

G
T
C
C
C
A
G
C
A
C
A
A
A
A
T
G
C
T
C
G
G
C
A
C
C
A
T
G
T
A
G
T
C
A
A
G
C
C
T
G
G
C
G
C
G
G
A
G
T
G
C
G
C
G
G
G
T
C
C
G
A
G
G
G
C
C
C
G
G
C
A
G
A
T
A
G
C
T
A
T
A
G
A
G
T
A
T
C
A
G
A
G
A
G
C
A
C
T
T
T
A
T
A
G
A
G
T
T
G
C
T
G
A
C
T
G
T
C
A
C
G
G
A
C
A
T
G
T
C
C
T
G
C
A
C
T
T
G
T
T
C
C
G
A
C
C
C
C
C
C
A
G
G
T
G
C
G
C
A
G
C
T
T
G
G
C
C
C
G
C
G
T
C
A
G
A
A
C
C
T
C
T
T
T
C
A
C
A
G
T
C
T
C
C
A
C
T
T
C
G
T
C
C
G
C
G
T
G
G
C
G
C
A
T
C
G
C
C
C
T
C
G
G
C
G
A
C
C
C
C
C
G
C
G
C
C
C
C
A
A
G
C
C
C
C
C
C
C
G
T
A
G
T
G
T
A
G
G
G
C
C
C
A
C
G
G
T
T
C
C
A
G
C
G
A
G
G
A
C
C
C
G
C
G
C
G
T
T
G
G
C
A
C
G
A
T
C
A
C
T
A
G
A
C
A
C
C
C
A
G
G
G
C
T
G
C
C
C
G
C
G
C
C
G
G
G
G
C
T
C
C
G
C
T
G
C
A
G
G
G
C
T
C
C
G
G
T
G
T
G
G
C
C
G
C
G
T
G
C
C
C
A
G
G
C
A
C
T
C
G
G
T
G
C
G
T
T
G
A
C
T
C
A
A
C
G
C
G
T
G
T
A
A
G
G
C
G
G
G
C
T
C
C
G
A
T
A
A
C
G
C
C
C
C
G
C
G
C
G
G
A
C
A
G
T
A
G
T
G
C
C
T
C
G
G
G
T
G
C
G
A
G
G
C
G
A
C
G
G
C
A
G
C
G
A
C
G
G
T
T
A
A
T
G
A
G
C
T
C
C
G
G
C
C
C
G
C
C
G
C
G
A
G
C
C
T
G
A
C
G
C
A
G
T
C
T
T
T
T
C


A
G
T
G
A
A
G
G
A
C
A
G
G
G
C
A
C
G
G
C
T
T
C
G
T
C
G
A
C
T
C
T
G
G
T
A
C
C
C
C
G
C
C
C
A
G
G
T
G

T
G
C
C
G
T
G
G
G
C
G
T
A
A
G
A
C
C
G
A
G
C
C
T
C
A
T
G
C
T
G
G
G
A
A
G
A
C
C
T
C
C
A
G
A
C
T
G
T
A
T
G
T
G
C
A
A
T
A
A
A
T
A
G
A
A
T
C
A
A
G
A
A
C
A
A
G
A
C
C
A
C
C
C
G
A
A
C
A
A
G
G
C
C
A
A
C
C
T
G
T
T
G
A
C
G
A
T
T
G
C
A
T
T
T
T
G
G
G
G
G
T
A
T
A
C
C
C
C
C
A
T
A
C
T
G
G
G
C
A
C
G
A
C
C
G
C
C
A
G
C
T
C
A
A
G
G
T
A
T
A
G
G
A
G
C
G
T
C
C
T
C
C
G
G
T
C
G
C
A
G
G
G
C
A
A
G
G
C
C
C
G
G
T
A
T
G
C
G
G
T
G
T
G
A
C
G
C
G
G
C
C
G
G
A
C
A
G
T
G
G
G
A
G
T
T
C
G
G
A
C
C
G
C
C
A
C
T
A
C
T
C
G
G
C
G
C
G
C
C
C
C
G
G
G
A
T
G
G
G
A
A
C
T
G
C
G
T
C
T
G
T
C
G
C
C
G
G
G
G
C
A
T
G
G
C
A
G
A
G
C
T
C
C
G
G
G
G
C
G
G
C
G
C
G
G
G
T
A
A
C
C
G
G
C
G
C
C
T
T
A
G
C
G
T
A
G
C
A
T
A
C
C
G
T
C
G
G
G
G
G
C
G
G
A
T
C
T
C
G
G
G
G
G
G
G
G
C
A
G
C
A
T
A
G
A
G
C
A
G
G
T
A
A
G
A
T
G
G
T
G
G
C
T
C
C
T
A
G
T
G
C
A
A
T
G
C
C
A
T
G
C
G
T
C
C
A
C
C
G
T
G
T
A
G
G
A
G
G
C
G
A
G
T
C
G
C
T
G
G
G
C
G
T
C
C
G
C
G
T
A
G
C
C
C
G
G
C
A
G
C

T
C
G
C
G
C
T
G
C
C
C
G
G
C
C
A
T
T
G
C
T
C
A
A
G
G
A
C
A
T
G
C
G
C
C
G
T
C
C
C
G
G
A
G
G
C
G
C
A
T
A
C
A
G
T
G
A
T
A
G
A
A
C
A
A
C
G
T
A
A
G
T
A
A
A
A
A
C
C
C
A
G
G
C
G
T
T
T
G
C
T
G
T
T
T
T
T
C
C
C
T
A
G
G
C
A
A
C
C
C
A
C
G
G
G
C
A
A
G
C
A
A
A
A
A
T
C
G
C
A
A
G
C
C
C
G
G
C
T
C
T
T
C
G
T
T
T
A
C
T
A
T
C
A
C
G
G
C
A
G
A
T
T
A
G
G
G
A
C
A
T
C
A
G
A
A
T
A
T
C
G
G
T
C
C
G
A
A
C
A
A
G
T
T
T
G
C
C
C
C
T
C
C
T
G
C
A
A
T
A
C
G
A
C
G
G
T
A
G
C
C
C
C
C
C
C
C
T
T
C
C
C
C
C
C
G
G
T
C
C
G
A
G
C
C
G
C
A
C
A
T
G
T
A
C
A
C
A
G
G
A
G
G
A
G
C
A
A
A
A
A
T
C
A
C
C
T
G
C
A
G
G
T
G
C
T
A
C
A
G
G
A
G
G
G
A
A
T
G
C
C
A
A
T
T
G
C
A
C
G
C
G
C
A
G
A
T
A
G
G
T
C
C
C
C
C
C
C
G
G
G
C
G
A
A
C
C
G
C
C
A
T
T
T
A
T
C
C
C
C
G
C
C
C
A
G
G
C
G
C
G
C
A
A
C
C
G
C
C
C
C
C
C
A
C
C
C
C
T
C
C
A
T
C
C
G
C
T
C
A
G
G
T
G
G
C
A
G
A
C
T
A
A
A
A
A
A
G
A
T
G
G
G
C
A
A
T
C
A
A
G
A
A
A
A
T
A
G
A
C
A
A
A
C
C
C
T
T
T
G
A
C
C
C
A
A
A
G
A
A
A
G
C
A
G
A
G
G
A
G
G
A
T
C
A
A
C
C
C
T
C
T
C
C
G
C
C
C
A
G
A
C
A
T
A
A
A
T
G
G
T
A
C
T
C
C
A
G
G
G


A
A
G
C
T
A
A
G
T
G
C
C
T
G
C
T
G
A
G
C
C
G
G
T
A
G
A
T
A
G
T
T
A
C
A
T
G
T
G
G
T
C
C
A
G
A
G
C
C
A
G
G
A
G
G
C
G
T
G
C
C
G
C
T
G
T
C
G
C
A
C
G
C
C
C
C
C
G
A
G
G
C
G
C
C
G
G
C
G
G
G
G
C
A
G
A
C
A
A
G
G
C
A
C
G
G
T
A
C
C
G
G
G
G
G
A
G
C
G
A
G
C
C
A
A
A
T
C
G
C
T
G
C
C
C
A
G
C
G
C
G
C
C
A
C
C
C
C
T
C
G
A
G
A
C
C
C
G
A
A
A
A
G
C
G
T
T
C
A
T
T
C
G
C
G
C
T
T
A
G
C
G
C
C
C
C
C
C
C
C
G
T
G
G
C
T
A
C
A
C
C
C
G
A
C
G
T
A
C
G
C
T
C
C
T
G
G
C
T
G
C
C
C
G
C
C
C
G
C
G
C
C
G
C
C
C
C
C
C
G
C
G
G
G
G
G
A
G
C
G
C
C
A
G
A
C
C
G
G
G
G
C
C
T
C
G
G
G
G
C
G
G
C
G
G
G
G
C
C
C
G
C
T
C
C
C
G
C
C
G
C
C
C
T
G
C
A
G
G
C
G
A
C
G
A
G
C
A
C
C
C
C
C
G
C
C
G
G
G
C
A
C
C
C
G
G
G
C
C
G
G
C
G
C
G
C
C
C
C
G
C
A
C
C
C
C
A
G
C
C
C
C
C
C
G
C
C
C
G
C
C
C
G
C
G
G
G
C
A
G
G
C
C
C
C
G
G
C
G
C
G
G
C
A
G
G
C
C
G
G
C
C
C
G
G
C
G
G
C
C
G
G
G
C
G
G
C
C
G
C
C
C
C
C
C
C
G
C
C
C
C
A
A
C
C
C
C
C
G
C
G
C
G
C
C
G
C
G
C
A
C
G
C
G
C
G
A
G
C
C
G
G
C
G
C
C
C
C
C
C
C
C
G
G
C
G
C
G
C
C
G
A
C
C
C
G
G
C
C
C
C
C
G
C
C
G
G
C
G
G
G
C
C
C
C
C
C
G
C
C


G
G
C
T
C
T
G
G
C
T
T
G
A
G
C
T
A
G
C
A
C
G
G
G
A
T
G
G
A
A
C
C
A
C
A
G
G
C
G
G
A
C
C
G
G
G
G
A
G
G
C
C
T
C
T
T
C
G
G
G
C
C
A
T
C
C
C
A
A
C
G
C
G
G
C
C
G
G
C
G
C
C
G
G
G
G
A
C
C
T
C
C
G
C
C
C
A
C
C
G
C
G
C
C
C
G
C
G
G
A
G
C
C
G
C
C
C
C
G
C
C
G
G
G
G
C
C
C
C
C
C
C
C
C
C
C
G
C
C
G
C
C
G
A
G
C
G
G
G
C
G
G
G
T
G
G
G
G
G
G
G
C
A
G
G
G
G
A
G
G
G
T
G
G
C
C
G
C
A
G
G
C
T
C
C
C
G
G
C
T
C
C
G
C
T
C
C
A
C
C
C
C
G
C
C
C
C
G
C
C
G
C
C
C
C
G
G
C
A
C
C
C
C
A
C
G
C
C
G
C
G
C
C
C
G
G
T
G
A
C
C
A
G
C
C
G
T
C
A
T
A
A
C
G
A
C
T
A
T
C
A
G
C
G
G
C
C
C
C
C
C
A
C
C
A
A
C
G
C
C
C
G
A
C
A
G
G
G
C
A
G
C
C
C
C
C
C
A
G
G
C
G
C
C
C
A
C
C
A
C
G
C
C
C
A
A
G
A
T
G
C
C
C
C
C
T
A
A
A
A
G
C
C
C
A
C
C
C
G
A
C
G
C
G
C
C
C
G
G
A
C
C
G
C
C
C
G
A
A
C
C
G
A
G
A
C
A
C
C
T
C
C
C
C
G
T
C
C
T
G
G
A
C
G
C
C
A
G
T
A
C
G
G
C
G
C
C
T
A
G
C
G
C
C
C
C
A
G
T
C
G
G
C
A
C
C
C
C
C
G
C
C
C
C
C
A
T
A
C
A
C
C
G
C
C
T
C
C
A
T
G
C
G
C
G
C
G
C
C
C
C
C
G
G
T
A
G
G
A
C
A
C
G
C
C
C
C
G
A
C
C
C
T
G
T
C
G
G
G
A
A
A
C
A
C
G
A
T
T
G
C
A
A
C
A
A
A
A
C
A


C
C
C
C
G
G
C
C
A
G
G
C
T
C
G
C
A
G
T
C
C
C
T
T
T
C
C
C
A
C
T
G
T
C
G
A
C
G
G
A
T
A
C
A
G
G
G
G
A
G
C
C
G
C
C
G
C
A
G
C
C
G
C
T
G
T
T
G
A
A
A
G
A
A
T
C
C
G
G
T
T
T
A
T
G
T
C
G
G
T
A
G
C
A
C
C
T
C
G
T
A
T
A
T
C
T
T
T
G
T
T
G
T
A
G
G
C
A
G
T
T
C
T
C
A
C
A
A
G
G
G
C
T
G
A
T
C
G
C
T
A
G
C
C
G
A
T
C
A
A
C
C
G
C
C
A
A
G
G
C
C
G
G
G
C
G
C
A
T
C
C
C
C
T
A
C
G
G
A
G
A
G
C
G
C
C
G
G
G
G
C
C
G
G
G
G
G
C
G
G
G
C
C
T
A
T
C
T
C
G
A
T
G
T
C
G
A
G
C
T
A
C
C
G
A
T
A
C
G
G
C
C
C
C
G
C
C
C
C
A
G
C
C
C
C
C
A
C
G
C
C
C
C
C
C
C
C
C
G
C
G
C
C
C
G
C
G
C
C
C
C
C
C
C
C
G
C
C
G
C
A
C
C
G
C
C
C
G
A
A
G
G
C
A
C
G
C
C
C
C
G
C
C
A
A
A
G
G
G
T
C
C
G
C
C
G
C
T
T
G
C
A
G
T
G
A
C
C
A
G
A
A
A
A
C
A
T
T
G
T
A
C
C
G
T
G
A
T
T
A
G
T
G
G
G
T
C
G
G
A
G
G
C
C
C
T
G
G
G
C
C
A
C
G
G
G
A
C
T
G
G
A
C
C
A
G
G
C
C
G
T
A
C
C
C
C
G
G
C
T
G
G
C
A
A
A
G
A
A
T
A
C
T
C
T
A
G
T
T
A
G
G
T
C
A
T
A
A
A
C
G
C
T
T
T
A
C
G
C
G
C
G
G
T
G
C
G
C
G
G
C
G
G
A
C
G
T
G
G
G
G
T
C
C
T
G
A
C
G
G
G
A
C
C
G
G
G
G
G
C
T
C
C
C
C
C
C
C
G
C
T
C
A
T
T
G
T
G
T
G


C
G
T
G
G
G
C
C
C
T
G
C
C
C
C
C
C
G
G
C
C
G
G
C
G
T
G
C
G
A
C
G
G
C
C
G
C
C
T
C
C
T
C
C
C
C
G
A
A
G
C
A
C
A
T
T
G
C
G
C
A
G
G
C
G
C
C
G
G
G
T
G
A
A
T
A
A
C
G
G
G
C
G
G
C
T
G
G
C
A
A
C
C
C
G
A
C
G
C
C
C
G
G
G
G
G
G
A
C
T
C
G
C
C
T
G
G
C
G
A
G
C
G
C
T
C
A
T
G
G
T
G
C
C
G
C
C
C
G
C
G
C
C
G
C
C
C
C
C
C
C
C
G
G
C
G
T
G
C
G
T
G
T
C
A
A
T
A
C
C
A
T
G
G
G
C
C
T
A
T
G
G
C
A
A
A
C
T
T
C
C
A
G
C
A
T
C
G
G
G
C
G
C
T
G
T
C
A
C
T
A
G
C
A
C
T
G
G
T
C
G
T
G
G
G
C
C
G
A
A
A
C
C
G
C
C
T
G
C
C
G
A
T
C
C
C
C
C
C
C
G
A
A
C
C
G
C
A
C
C
G
C
C
T
A
C
G
A
A
A
A
G
A
G
C
C
C
G
C
C
G
C
C
A
A
T
G
G
C
C
G
G
T
A
C
C
C
A
C
C
C
A
G
C
A
G
A
T
A
A
T
A
G
C
A
A
C
C
G
G
G
G
C
G
C
C
C
G
A
G
C
T
G
G
G
G
G
C
T
G
C
A
A
A
A
G
A
C
C
C
G
C
G
C
G
C
G
C
T
C
C
C
C
C
C
T
A
T
G
G
A
G
C
A
G
A
A
G
A
C
T
C
A
A
C
C
G
G
A
C
G
A
C
A
T
C
C
A
C
G
T
T
G
C
T
C
C
C
G
G
C
T
G
C
G
T
T
A
G
T
T
C
C
C
A
A
G
T
C
A
C
T
C
C
C
A
C
G
G
T
C
C
A
G
T
G
T
C
A
T
C
T
A
G
C
A
A
C
C
T
C
A
C
A
G
C
C
C
C
G
G
A
C
T
G
G
T
A
A
A
G
A
A
C
T
C
G
T
A
G
C
C
T
T
A
T
A
G
G
C


T
G
C
A
C
C
C
G
G
G
C
T
G
G
G
C
G
G
A
G
C
C
C
G
C
G
G
G
G
C
C
C
G
T
G
T
G
C
G
C
G
C
G
G
C
C
C
C
A
G
G
C
T
G
C
A
C
C
G
G
A
C
G
G
C
C
G
G
C
G
C
G
G
C
C
G
C
G
C
A
A
G
C
G
T
T
C
C
G
G
G
G
C
G
C
C
C
C
G
C
C
C
G
G
C
C
T
C
G
C
G
C
G
G
C
G
A
A
C
G
C
G
C
A
C
C
C
G
T
C
G
C
C
T
C
G
T
T
G
T
C
T
C
C
G
T
C
C
C
G
C
A
T
G
C
G
C
G
G
C
C
C
C
C
C
A
C
T
G
C
C
G
G
C
A
C
C
C
C
C
A
C
A
C
G
G
G
G
T
C
C
A
C
T
C
G
G
C
A
G
C
T
C
A
G
T
T
G
A
A
G
G
C
C
G
C
G
C
T
G
G
C
G
G
T
G
C
C
A
G
C
T
C
G
G
G
G
C
C
C
C
C
A
G
G
T
T
C
T
T
G
C
C
C
C
C
C
G
G
C
A
T
C
C
G
G
G
C
G
G
C
C
G
G
C
T
G
G
G
C
C
C
C
G
G
C
C
C
G
G
A
C
G
C
C
C
G
C
C
C
G
G
C
G
T
A
G
G
A
G
G
G
G
G
G
G
G
G
T
T
C
T
T
C
T
T
G
C
A
C
C
C
C
G
G
C
C
C
C
G
G
A
G
G
C
A
A
G
G
C
G
G
C
C
C
C
G
C
G
A
A
G
C
C
T
C
C
G
C
C
C
G
T
G
A
G
C
C
G
G
G
G
A
C
G
G
G
A
G
G
A
G
C
T
T
C
A
C
G
G
T
C
C
C
C
G
C
C
C
G
G
C
G
C
G
C
A
C
C
G
G
G
A
G
G
G
G
C
C
C
G
G
G
G
C
T
G
A
C
C
C
C
G
G
C
C
G
G
C
A
T
G
T
G
A
G
G
G
A
A
G
G
G
C
C
T
A
A
G
G
A
G
T
G
G
C
A
G
G
A
C
G
G
A
G
C
G
C
A
T
A
G
C
C
C
C
G
G
C
C


In [19]:
for i in range(len(Sequence)):
    seq = Sequence[i][::-1]
    seq = Seq(seq).complement()
    seq = str(seq)
    
    input_sequence = seq[:(Negative.loc[i,("SNPposition")]-1)] + str(Seq(Alt[i]).complement()) + seq[(Negative.loc[i,("SNPposition")]):]
    
    x = one_hot_encode('N'*(context//2) + input_sequence + 'N'*(context//2))[None, :]
    y = np.mean([models[m].predict(x) for m in range(5)], axis=0)
    m6AAI_prob = y[0, :, 1]
    Probability = pd.DataFrame({'Probability':m6AAI_prob})
    Probability.sort_index(inplace=True, ascending=False)
    Probability = Probability.reset_index(drop = True)
    
    df = pd.DataFrame(np.random.randn((Negative.loc[i,"GeneLength"]), 3))
    df.columns = ["name", "chrom", "strand"]
    df["name"] = Negative.loc[i,"SNPID"]
    df["chrom"] = Negative.loc[i,"chrom"]
    df["strand"] = Negative.loc[i,"strand"]
    
    list = range(Negative.loc[i,"txStart"], Negative.loc[i,"txEnd"])
    Start = pd.DataFrame(list, columns=["Start"])
    df = pd.concat([df, Start], axis=1)    
    df["End"] = df["Start"]
    df = pd.concat([df, Probability], axis=1)    
    
    df.columns = ["name", "chrom", "strand", "Start", "End", "Probabilty"]
    df = df.head(Negative.loc[i,"LastExonLength"])
    df.to_csv("./m6AQTL/{}.bed".format(Negative.loc[i,"SNPID"]), sep="\t", index=False)

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  if sys.path[0] == '':
